## Overview

(drafting)

#### How this application works

(drafting)

## Prerequisites

Before you start processing this notebook, some prerequisites need to be completed.

* Set up your AWS Panorama Appliance - [middle click to open document](https://docs.aws.amazon.com/panorama/latest/dev/gettingstarted-setup.html)
* Create an IAM Role for your application - [middle click to open document](https://docs.aws.amazon.com/panorama/latest/dev/permissions-application.html)
* Install "panorama-cli" tool [middle click to open document](https://docs.aws.amazon.com/panorama/latest/dev/gettingstarted-deploy.html#gettingstarted-deploy-prerequisites)

## Import libraries and define configurations

First step is to import all libraries needed.

In [ ]:
import sys
import os
import time
import tarfile
import json
import uuid

import boto3

sys.path.insert( 0, os.path.abspath( "../common/test_utility" ) )
import panorama_test_utility

You need to specify some information specific to your environment.

In [ ]:
account_id = boto3.client("sts").get_caller_identity()["Account"]
region_name = boto3.session.Session().region_name

print( "account_id :", account_id )
print( "region_name :", region_name )

# Following configurations are required when you use real hardware, 
# thus can be any dummy strings when you use only Test Utility.
device_id = input("Device Id (format : device-*)").strip()

## Import application

With "panorama-cli import-application" command, replacing placeholder information in application files. This step essentially replace placeholder ("123456789012") with your aws account id.

In [ ]:
!cd ./inbound_network_app/ && panorama-cli import-application

## Prepare business logic

#### Preview python source code
Next step is to build a business logic container. This application's business logic consists of single python source code. Let's preview it.

In [ ]:
panorama_test_utility.preview_text_file( f"./inbound_network_app/packages/{account_id}-inbound_network_code-1.0/src/app.py" )

#### Test run the business logic with test-utility

Let's run the application with Test Utility **Run** command. In this sample, we don't use any model, so model compilation step is not needed.

To iterate the development, you can edit the python source code with your prefered text editor (or even within Jupyter environment), and re-run the application.

In [ ]:
# Run the application with test-utility.

%run ../common/test_utility/panorama_test_utility_run.py \
--app-name inbound_network_app \
--code-package-name inbound_network_code \
--py-file ./inbound_network_app/packages/{account_id}-inbound_network_code-1.0/src/app.py

#### Build application logic container

With "panorama-cli build-container" command, building a container image, and add it into the "inbound_network_code" package.

This step takes long time (5~10 mins), and because it is using %%capture magic command, you don't see progress during the process. Please wait.

In [ ]:
%%capture captured_output
# FIXME : without %%capture, browser tab crashes because of too much output from the command.

!cd ./inbound_network_app && panorama-cli build-container \
    --container-asset-name code \
    --package-path packages/{account_id}-inbound_network_code-1.0

In [ ]:
stdout_lines = captured_output.stdout.splitlines()
stderr_lines = captured_output.stderr.splitlines()
print("     :")
print("     :")
for line in stdout_lines[-30:] + stderr_lines[-30:]:
    print(line)

## Package application (upload locally prepared packages onto Cloud)

Now you have prepared both model packages and code package locally. Let's upload those packages to the cloud with "panorama-cli package-application" command.

In [ ]:
!cd ./inbound_network_app && panorama-cli package-application

## Deploy application to the device programatically

Once you uploaded the packages to the cloud, you can create an application instance on your device. You need to specify a manifest file.

#### Preview manifest file("graph.json"), and override-manifest file("override.json")

In [ ]:
panorama_test_utility.preview_text_file( "./inbound_network_app/graphs/inbound_network_app/graph.json" )

#### Deploy the app using the manifest file

In order to create an application instance, this notebook uses boto3's "panorama" client and create_application_instance() API. (It is also possible to use "aws panorama create-application-instance" command instead.)

In [ ]:
# create a boto3 client to access Panorama service
# FIXME : not using AWS_REGION here, because panorama-cli uses only default region currently.
panorama_client = boto3.client("panorama")

In [ ]:
def deploy_application( application_name, manifest_filename, override_filename=None ):

    def get_payload_from_json( filename ):
        with open( filename ) as fd:
            
            s = fd.read()
            
            # validating JSON format and making it compact, by loading and dumping, 
            payload = json.dumps(json.loads(s))
            
            return payload

    manifest_payload = get_payload_from_json( manifest_filename )
    
    params = {
        "Name" : application_name,
        "DefaultRuntimeContextDevice" : device_id,
        "ManifestPayload" : {"PayloadData":manifest_payload},
    }
    
    if override_filename:
        override_payload = get_payload_from_json( override_filename )
        params["ManifestOverridesPayload"] = {"PayloadData":override_payload}
    
    response = panorama_client.create_application_instance( ** params )
        
    return response

In [ ]:
application_name = "inbound_network_notebook_" + str(uuid.uuid4())[:8]

response = deploy_application(
    application_name = application_name,
    manifest_filename = "./inbound_network_app/graphs/inbound_network_app/graph.json",
)

application_instance_id = response["ApplicationInstanceId"]

response

In [ ]:
application_name

#### Wait for deployment completion

Application instance creation has been triggered. This notebook checks the progress by calling describe_application_instance() API periodically. Please confirm that you see "DEPLOYMENT_SUCCEEDED" status at the end.

In [ ]:
def wait_deployment( application_instance_id ):
    
    progress_dots = panorama_test_utility.ProgressDots()    
    while True:
        app = panorama_client.describe_application_instance( ApplicationInstanceId = application_instance_id )
        progress_dots.update_status( "%s (%s)" % (app["Status"], app["StatusDescription"]) )
        if app["Status"] not in ( "DEPLOYMENT_PENDING", "DEPLOYMENT_REQUESTED", "DEPLOYMENT_IN_PROGRESS" ):
            break
        time.sleep(60)

wait_deployment( application_instance_id )

#### Visit CloudWatch Logs to check logs from the application instance

If you saw "DEPLOYMENT_SUCCEEDED" status, the application started to run on your device. Application logs are uploaded to CloudWatch Logs. Let's get the URL of CloudWatch Logs management console. "console_output" is the log stream your Python code's stdout/stderr are redirected to.

In [ ]:
logs_url = panorama_test_utility.get_logs_url( region_name, device_id, application_instance_id )
print( "CloudWatch Logs URL :" )
print( logs_url )